In [107]:
import numpy as np

In [108]:
import pandas as pd

In [109]:
weather = pd.read_csv('weather.csv')
df = weather.copy()

In [110]:
df = df.iloc[:,1:len(df.columns)]

In [111]:
df.tail()
df.columns

Index(['precipitation', 'temp_max', 'temp_min', 'wind', 'weather'], dtype='object')

One Hot Encoding

In [112]:
df = pd.get_dummies(df, columns=['weather'])
df.tail()

,precipitation,temp_max,temp_min,wind,weather_drizzle,weather_fog,weather_rain,weather_snow,weather_sun
1456,8.6,4.4,1.7,2.9,0,0,1,0,0
1457,1.5,5.0,1.7,1.3,0,0,1,0,0
1458,0.0,7.2,0.6,2.6,0,1,0,0,0
1459,0.0,5.6,-1.0,3.4,0,0,0,0,1
1460,0.0,5.6,-2.1,3.5,0,0,0,0,1


Dataframe -> Numpy Array

In [113]:
X = df.iloc[:,:4].values
Y = df.iloc[:,4:9].values

train and test datas

In [114]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.2, random_state=0)

Waow, I found new stuff

In [115]:
# from lazypredict.Supervised import LazyClassifier
# clf = LazyClassifier(verbose = 0, ignore_warnings=True, custom_metric=None)
# models,predictions = clf.fit(x_train, x_test, y_train, y_test)
# print(models)

I'll applied the above one in another project, I could not handle it here. Maybe that's why it is multi-classificatin problem

fit KNN model

In [116]:
# from sklearn.neighbors import KNeighborsClassifier
# knn = KNeighborsClassifier(n_neighbors=7)
# knn.fit(x_train, y_train)

predict through KNN

In [117]:
# y_pred = knn.predict(x_test)

In [118]:
# from sklearn.metrics import r2_score
# r2_score(y_test, y_pred)

## ANN

Model Scaling: Minmaxscaler

In [119]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(x_train)
X_test = scaler.fit_transform(x_test)

In [120]:
from keras.models import Sequential
from keras.layers import Dense

In [121]:
classifier = Sequential()

classifier.add(Dense(
    units = 6,
    kernel_initializer = 'uniform',
    activation = 'relu',
    input_dim = 4
))

classifier.add(Dense(
    units = 8,
    kernel_initializer = 'uniform',
    activation = 'relu'
))

classifier.add(Dense(
    units = 10,
    kernel_initializer = 'uniform',
    activation = 'relu'
))


classifier.add(Dense(
    units = 5,
    kernel_initializer = 'uniform',
    activation = 'softmax',
))

In [122]:
classifier.compile(optimizer='adam',
                   loss = 'categorical_crossentropy',
                   metrics = ['accuracy'])

In [123]:
classifier.fit(X_train,y_train,epochs = 100, batch_size=32, verbose=1)

Epoch 1/100
37/37 [==============================] - 2s 4ms/step - loss: 1.5907 - accuracy: 0.4161
Epoch 2/100
37/37 [==============================] - 0s 4ms/step - loss: 1.5357 - accuracy: 0.4435
Epoch 3/100
37/37 [==============================] - 0s 3ms/step - loss: 1.4262 - accuracy: 0.4435
Epoch 4/100
37/37 [==============================] - 0s 4ms/step - loss: 1.2384 - accuracy: 0.4435
Epoch 5/100
37/37 [==============================] - 0s 3ms/step - loss: 1.1097 - accuracy: 0.4435
Epoch 6/100
37/37 [==============================] - 0s 4ms/step - loss: 1.0838 - accuracy: 0.4289
Epoch 7/100
37/37 [==============================] - 0s 3ms/step - loss: 1.0805 - accuracy: 0.4418
Epoch 8/100
37/37 [==============================] - 0s 4ms/step - loss: 1.0792 - accuracy: 0.4435
Epoch 9/100
37/37 [==============================] - 0s 4ms/step - loss: 1.0784 - accuracy: 0.4195
Epoch 10/100
37/37 [==============================] - 0s 4ms/step - loss: 1.0772 - accuracy: 0.4324
Epoch 11/

In [124]:
y_pred = classifier.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy = classifier.evaluate(X_test, y_test, verbose=0)
print("Accuracy: {:.2f}%".format(accuracy[1] * 100))


10/10 [==============================] - 0s 2ms/step
Accuracy: 74.40%


So multi-classification prediction problem has done here, the accuracy which we reach is about 80%. We could have better solution with ML algorithms such as Random Forest, SVC. However we applied neural networks, so there it goes.

Okay, let's try to solve the same problem with the XGBoost which is comman approach of ML 

## XGBoost

In [125]:
import xgboost as xgb

Let's use DMatrix which is particular dataframe/matrix form of XGB

In [126]:
dm_train = xgb.DMatrix(data = X_train, label = y_train)
dm_test = xgb.DMatrix(data = X_test, label = y_test)

In [127]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)
print(y_train)
y_pred = xgb_model.predict(X_test)

[[0 0 1 0 0]
 [0 0 0 0 1]
 [0 0 1 0 0]
 ...
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 1 0 0]]


Let's check achievement of model XGB

In [128]:
accuracy_score(y_test, y_pred)

0.7064846416382252

Okay, np time to give in. Let's try model tuning.

### XGB Model Tuning

In [129]:
from sklearn.model_selection import GridSearchCV

In [130]:
xgb_params = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [50,100,200,500],
    'max_depth':[3,4,5,6],
    'min_samples_split': [2.5,5],
    'subsample':[0.6,0.8,1.0]
}

In [132]:
xgb = XGBClassifier()
xgb_cv_model = GridSearchCV(xgb, xgb_params, cv=3, n_jobs=-1, verbose=1)

In [133]:
xgb_cv_model.fit(X_train, y_train) #3 minute

Fitting 3 folds for each of 288 candidates, totalling 864 fits


KeyboardInterrupt: 

In [ ]:
xgb_cv_model.best_params_

{'learning_rate': 0.01,
 'max_depth': 3,
 'min_samples_split': 2.5,
 'n_estimators': 200,
 'subsample': 0.6}

In [ ]:
xgb_tuned = XGBClassifier(
    learning_rate  = 0.01,
    max_depth = 3,
    min_samples_split = 2.5,
    n_estimators = 200,
    subsample = 0.6
)
xgb_tuned = xgb.fit(X_train, y_train)

In [ ]:
y_pred = xgb_model.predict(X_test)
accuracy_score(y_test, y_pred)
# I am crying 

0.7064846416382252

Okay, here we go try Light GBM

In [ ]:
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier()

# lgbm.fit(X_train,y_train)

ValueError: y should be a 1d array, got an array of shape (1168, 5) instead.

We can not use dummy variables which cause 2D array in LGBM

In [ ]:
y_train_no_dummies = np.argmax(y_train,axis=1)
y_test_no_dummies = np.argmax(y_test,axis=1)
# The argmax function can be used to convert the dummy variables back to their original categorical form.

In [ ]:
lgbm.fit(X_train, y_train_no_dummies)

LGBMClassifier()

In [ ]:
y_pred = lgbm.predict(X_test)
accuracy_score(y_test_no_dummies, y_pred)

0.7508532423208191

Better than XGB, hah?

### LightGBM Model Tuning

In [ ]:
?lgbm

Type:           LGBMClassifier
String form:    LGBMClassifier()
File:           c:\users\anerg\appdata\local\programs\python\python310\lib\site-packages\lightgbm\sklearn.py
Docstring:      LightGBM classifier.
Init docstring:
Construct a gradient boosting model.

Parameters
----------
boosting_type : str, optional (default='gbdt')
    'gbdt', traditional Gradient Boosting Decision Tree.
    'dart', Dropouts meet Multiple Additive Regression Trees.
    'goss', Gradient-based One-Side Sampling.
    'rf', Random Forest.
num_leaves : int, optional (default=31)
    Maximum tree leaves for base learners.
max_depth : int, optional (default=-1)
    Maximum tree depth for base learners, <=0 means no limit.
learning_rate : float, optional (default=0.1)
    Boosting learning rate.
    You can use ``callbacks`` parameter of ``fit`` method to shrink/adapt learning rate
    in training using ``reset_parameter`` callback.
    Note, that this will ignore the ``learning_rate`` argument in training.
n_e

In [ ]:
lgbm_params = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [50,100,200,500],
    'max_depth':[3,4,5,6],
    'min_child_samples': [5,10,20],
    'subsample':[0.6,0.8,1.0]
}

In [ ]:
lgbm_cv_model = GridSearchCV( lgbm, lgbm_params, cv=3, n_jobs = -1, verbose=2)

In [ ]:
lgbm_cv_model.fit(X_train, y_train_no_dummies)
# it does not suppose to get longer but it did

Fitting 3 folds for each of 432 candidates, totalling 1296 fits


GridSearchCV(cv=3, estimator=LGBMClassifier(), n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [3, 4, 5, 6],
                         'min_child_samples': [5, 10, 20],
                         'n_estimators': [50, 100, 200, 500],
                         'subsample': [0.6, 0.8, 1.0]},
             verbose=2)

In [ ]:
lgbm_cv_model.best_params_

{'learning_rate': 0.01,
 'max_depth': 4,
 'min_child_samples': 5,
 'n_estimators': 100,
 'subsample': 0.6}

In [ ]:
lgbm = LGBMClassifier(learning_rate=0.01,
                      max_depth=4,
                      min_child_samples=5,
                      n_estimators=100,
                      subsample=0.6)

In [ ]:
lgbm_tuned = lgbm.fit(X_train, y_train_no_dummies)
y_pred = lgbm_tuned.predict(X_test)
accuracy_score(y_test_no_dummies, y_pred)

0.7849829351535836

At least it got better unlike XGB 

### CatBoost

In [134]:
from catboost import CatBoostClassifier

In [135]:
cat = CatBoostClassifier()
cat.fit(X_train, y_train_no_dummies)

Learning rate set to 0.079756
0:	learn: 1.4177855	total: 9.35ms	remaining: 9.34s
1:	learn: 1.2786818	total: 18ms	remaining: 8.96s
2:	learn: 1.1684219	total: 24.7ms	remaining: 8.2s
3:	learn: 1.0781171	total: 30.5ms	remaining: 7.6s
4:	learn: 1.0040766	total: 35.7ms	remaining: 7.1s
5:	learn: 0.9420286	total: 39.3ms	remaining: 6.51s
6:	learn: 0.8896643	total: 42.6ms	remaining: 6.04s
7:	learn: 0.8448219	total: 47.8ms	remaining: 5.92s
8:	learn: 0.8087153	total: 51.3ms	remaining: 5.65s
9:	learn: 0.7739427	total: 55.1ms	remaining: 5.45s
10:	learn: 0.7467268	total: 59.3ms	remaining: 5.33s
11:	learn: 0.7211815	total: 63.5ms	remaining: 5.23s
12:	learn: 0.6964396	total: 67.1ms	remaining: 5.1s
13:	learn: 0.6740842	total: 70.4ms	remaining: 4.96s
14:	learn: 0.6542234	total: 74.4ms	remaining: 4.88s
15:	learn: 0.6361684	total: 78ms	remaining: 4.8s
16:	learn: 0.6189438	total: 82.3ms	remaining: 4.76s
17:	learn: 0.6073760	total: 85.8ms	remaining: 4.68s
18:	learn: 0.5935085	total: 110ms	remaining: 5.67s
19

In [136]:
y_pred = cat.predict(X_test)
accuracy_score(y_test_no_dummies, y_pred)

0.7508532423208191

it seems exactly same result with lgbm

#### CatBoost Model Tuning

In [137]:
catb_params = {
    'learning_rate': [0.01, 0.05, 0.1],
    'iterations': [200,500],
    'depth': [3,5,8]
}

catb_cv_model = GridSearchCV(cat, catb_params, cv = 3, n_jobs = -1, verbose = 2)
catb_cv_model.fit(X_train, y_train_no_dummies)
catb_cv_model.best_params_ 

Fitting 3 folds for each of 18 candidates, totalling 54 fits
0:	learn: 1.3603999	total: 1.43ms	remaining: 285ms
1:	learn: 1.1935247	total: 3.18ms	remaining: 315ms
2:	learn: 1.0739068	total: 5.09ms	remaining: 334ms
3:	learn: 0.9835130	total: 7.07ms	remaining: 346ms
4:	learn: 0.9113674	total: 8.66ms	remaining: 338ms
5:	learn: 0.8550128	total: 9.91ms	remaining: 320ms
6:	learn: 0.8011870	total: 11.3ms	remaining: 311ms
7:	learn: 0.7585354	total: 12.9ms	remaining: 309ms
8:	learn: 0.7206951	total: 14.4ms	remaining: 306ms
9:	learn: 0.6906870	total: 15.9ms	remaining: 303ms
10:	learn: 0.6652195	total: 17.5ms	remaining: 300ms
11:	learn: 0.6432697	total: 18.9ms	remaining: 296ms
12:	learn: 0.6223886	total: 21ms	remaining: 302ms
13:	learn: 0.6046502	total: 22.6ms	remaining: 300ms
14:	learn: 0.5881799	total: 23.9ms	remaining: 295ms
15:	learn: 0.5757754	total: 26.9ms	remaining: 309ms
16:	learn: 0.5649614	total: 28.4ms	remaining: 306ms
17:	learn: 0.5573782	total: 30ms	remaining: 304ms
18:	learn: 0.5480

{'depth': 3, 'iterations': 200, 'learning_rate': 0.1}

In [138]:
cat = CatBoostClassifier(learning_rate= 0.1,
                      iterations = 200,
                      depth = 3)

In [139]:
cat_tuned = cat.fit(X_train, y_train_no_dummies)
y_pred = cat_tuned.predict(X_test)
accuracy_score(y_test_no_dummies, y_pred)

0:	learn: 1.3603999	total: 1.65ms	remaining: 328ms
1:	learn: 1.1935247	total: 3.4ms	remaining: 337ms
2:	learn: 1.0739068	total: 4.84ms	remaining: 318ms
3:	learn: 0.9835130	total: 6.79ms	remaining: 333ms
4:	learn: 0.9113674	total: 8.41ms	remaining: 328ms
5:	learn: 0.8550128	total: 10.2ms	remaining: 330ms
6:	learn: 0.8011870	total: 11.9ms	remaining: 328ms
7:	learn: 0.7585354	total: 13.5ms	remaining: 324ms
8:	learn: 0.7206951	total: 15ms	remaining: 318ms
9:	learn: 0.6906870	total: 17.5ms	remaining: 333ms
10:	learn: 0.6652195	total: 19.4ms	remaining: 334ms
11:	learn: 0.6432697	total: 20.8ms	remaining: 325ms
12:	learn: 0.6223886	total: 22.3ms	remaining: 320ms
13:	learn: 0.6046502	total: 23.8ms	remaining: 316ms
14:	learn: 0.5881799	total: 25.2ms	remaining: 311ms
15:	learn: 0.5757754	total: 26.4ms	remaining: 304ms
16:	learn: 0.5649614	total: 27.7ms	remaining: 298ms
17:	learn: 0.5573782	total: 29.5ms	remaining: 298ms
18:	learn: 0.5480477	total: 32.5ms	remaining: 309ms
19:	learn: 0.5404586	tota

0.7952218430034129

We are gettting our model better step by step due to tring different models :)

Almost same with the ANN